In [75]:
import os
import glob
import gdal
from scipy.ndimage import zoom
import subprocess

# Creating custom directories for downscaling/conversion

In [23]:
def create_directories(d1, d2, tiles, area):
    for tile in tiles:
        os.mkdir(area + tile + '/' + d1)
        os.mkdir(area + tile + '/' + d2)

In [42]:
tiles = ['T1', 'T2', 'T3', 'T4']
area  = 'Area_one/Tiles/'

d1 = 'D10m'
d2 = 'D20m'

In [25]:
create_directories(d1, d2, tiles, area)

# Downscaling and Conversion

In [27]:
def setGeo(geotransform, bgx, bgy, x_offset = 0):
    if x_offset == 0:
        x_offset = geotransform[1]
        y_offset = geotransform[5]
    else:
        x_offset = x_offset
        y_offset = -x_offset
    reset0 = geotransform[0] + bgx * geotransform[1]
    reset3 = geotransform[3] + bgy * geotransform[5]
    reset = (reset0, x_offset, geotransform[2],
             reset3, geotransform[4], y_offset)
    return reset

In [34]:
def downscale(in_dir, out_dir):
    files = glob.glob(in_dir + '*')
    files.sort()

    for file in files:
        f = gdal.Open(file, gdal.GA_ReadOnly)
        projection = f.GetProjection()
        geotransform = f.GetGeoTransform()

        f = f.ReadAsArray()

        new_geo = setGeo(geotransform, 0, 0, x_offset = 10)
        f = zoom(f, [2,2], order = 0, mode = 'nearest')
        fx, fy = f.shape

        file_name = file[-11:-4]
        file_dir = out_dir + file_name
        outdata = gdal.GetDriverByName('GTiff').Create(file_dir +'.tif', fy, fx, 1, gdal.GDT_UInt16)
        outdata.SetGeoTransform(new_geo)
        outdata.SetProjection(projection)
        outdata.GetRasterBand(1).WriteArray(f)
        outdata.FlushCache()
        outdata = None

In [35]:
for tile in tiles:
    input_dir = area + tile + '/' + 'R20m/'
    output_dir = area + tile + '/' + 'D20m/'

    print(input_dir, output_dir)

    downscale(input_dir, output_dir)

Area_one/Tiles/T1/R20m/ Area_one/Tiles/T1/D20m/
Area_one/Tiles/T2/R20m/ Area_one/Tiles/T2/D20m/
Area_one/Tiles/T3/R20m/ Area_one/Tiles/T3/D20m/
Area_one/Tiles/T4/R20m/ Area_one/Tiles/T4/D20m/


# Converting 10m bands to tiff format

In [36]:
def convert(in_dir, out_dir):
    files = glob.glob(in_dir + '*')
    files.sort()

    for file in files:
        f = gdal.Open(file, gdal.GA_ReadOnly)
        projection = f.GetProjection()
        geotransform = f.GetGeoTransform()

        f = f.ReadAsArray()
        fx, fy = f.shape

        file_name = file[-11:-4]
        file_dir = out_dir + file_name
        outdata = gdal.GetDriverByName('GTiff').Create(file_dir +'.tif', fy, fx, 1, gdal.GDT_UInt16)
        outdata.SetGeoTransform(geotransform)
        outdata.SetProjection(projection)
        outdata.GetRasterBand(1).WriteArray(f)
        outdata.FlushCache()
        outdata = None

In [37]:
for tile in tiles:
    input_dir = area + tile + '/' + 'R20m/'
    output_dir = area + tile + '/' + 'D20m/'

    print(input_dir, output_dir)

    convert(input_dir, output_dir)

Area_one/Tiles/T1/R20m/ Area_one/Tiles/T1/D20m/
Area_one/Tiles/T2/R20m/ Area_one/Tiles/T2/D20m/
Area_one/Tiles/T3/R20m/ Area_one/Tiles/T3/D20m/
Area_one/Tiles/T4/R20m/ Area_one/Tiles/T4/D20m/


# Layer Stacking 20m bands [02, 03, 04]

In [39]:
layers = ['02', '03', '04']
area = 'Area_one/Tiles/'

In [66]:
def combine(input_dir, output_dir, index):
    files = []
    for l in layers:
        fs = [f for f in glob.glob(input_dir + 'B*')]
        files += fs
    
    count = 0
    name = output_dir + 'Stacked_Raster_' + str(index)
    fz = len(files)

    for file in files:

        f = gdal.Open(file, gdal.GA_ReadOnly)
        projection = f.GetProjection()
        geotransform = f.GetGeoTransform()

        f = f.ReadAsArray()
        fx, fy = f.shape

        count = count + 1
        if count == 1:
            outdata = gdal.GetDriverByName('GTiff').Create(name + '.tif', fy, fx, fz,gdal.GDT_UInt16)
            outdata.SetGeoTransform(geotransform)
            outdata.SetProjection(projection)
        outdata.GetRasterBand(count).WriteArray(f)
        outdata.FlushCache() 
    outdata = None

In [67]:
index = 0
for tile in tiles:
    input_dir, output_dir = area + tile + '/' + d2 + '/', area + tile + '/'
    combine(input_dir, output_dir, index)

    input_dir, output_dir = area + tile + '/' + d1 + '/', area + tile + '/'
    combine(input_dir, output_dir, index) 

    index = index + 1

# Mosaing Rasters to create study area

In [72]:
tiles = ['T1', 'T2']

In [74]:
raster_files = []
for tile in tiles:
    files = glob.glob(area + tile + '/' + 'Stacked_Raster_*.tif')
    print(files)
    # cmd = 'gdal_merge.py -ot Int16 -o Area_one/Tiles/Main_Raster.tif'
    # subprocess.call(cmd.split()+files)

['Area_one/Tiles/T1/Stacked_Raster_0.tif']
['Area_one/Tiles/T2/Stacked_Raster_1.tif']
